럭키백의 확률

In [1]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [3]:
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [4]:
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy() # 넘파이 배열로 변환
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [7]:
fish_target = fish['Species'].to_numpy() # 문자는 숫자로

세트 분할

In [9]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

표준화

In [12]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

k neighbors -> 다중 분류

In [16]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
kn.score(train_scaled, train_target), kn.score(test_scaled, test_target)

(0.8907563025210085, 0.85)

In [18]:
kn.classes_

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

In [19]:
kn.predict(test_scaled[:5])

array(['Perch', 'Smelt', 'Pike', 'Perch', 'Perch'], dtype=object)

In [22]:
# proba = kn.predict_proba(test_scale[:5]) # proba : 확률을 출력
kn.predict_proba(test_scaled[:3])

array([[0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0.]])

로지스틱 회귀
- 양성 : 관심대상
- 선형회귀 + 시그모이드 -> 확률값 사용, 0.5보다 크면 양성으로 예측
- 0.5는 음성으로 예측

- 회귀 : 실수값 예측
- 분류 : 클래스 예측 <- 로지스틱 회귀

- 양성 클래스의 확률 : 시그모이드 함수의 결과
- 음성 클래스의 확률 1 - 결과

로지스틱 회귀로 이진 분류

In [26]:
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt') # boolean
train_bream_smelt = train_scaled[bream_smelt_indexes] # boolean indexing
target_bream_smelt = train_target[bream_smelt_indexes] # boolean indexing

In [27]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

LogisticRegression()

In [28]:
lr.predict(train_bream_smelt[:5])

array(['Bream', 'Smelt', 'Bream', 'Bream', 'Bream'], dtype=object)

In [30]:
lr.predict_proba(train_bream_smelt[:5])

array([[0.99759855, 0.00240145],
       [0.02735183, 0.97264817],
       [0.99486072, 0.00513928],
       [0.98584202, 0.01415798],
       [0.99767269, 0.00232731]])

In [31]:
lr.coef_, lr.intercept_

(array([[-0.4037798 , -0.57620209, -0.66280298, -1.01290277, -0.73168947]]),
 array([-2.16155132]))

In [32]:
decisions = lr.decision_function(train_bream_smelt[:5]) # 선형 함수의 결과값 , z.
# z를 phi로 바꾼게 predict_proba, class로 변경한 게 predict
print(decisions)

[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]


In [34]:
lr.decision_function(train_bream_smelt[:5]) # 위와 같은 결과

array([-6.02927744,  3.57123907, -5.26568906, -4.24321775, -6.0607117 ])

In [33]:
from scipy.special import expit
expit(decisions)

array([0.00240145, 0.97264817, 0.00513928, 0.01415798, 0.00232731])

로지스틱 회귀로 다중 분류
ex. 클래스 3개 bream. smelt, perch
- 이진 : bream = 양성, smelt&perch = 음성 등....
- 다중 : 알파의 역수로 규제

In [38]:
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)
lr.score(train_scaled, train_target), lr.score(test_scaled, test_target) # 훈련, 테스트 점수

(0.9327731092436975, 0.925)

In [39]:
lr.predict(test_scaled[:5])

array(['Perch', 'Smelt', 'Pike', 'Roach', 'Perch'], dtype=object)

In [40]:
lr.predict_proba(test_scaled[:5])

array([[7.24962119e-06, 1.35114379e-02, 8.41283507e-01, 3.14327385e-04,
        1.35660262e-01, 6.67136890e-03, 2.55184724e-03],
       [7.14996969e-09, 2.55552480e-03, 4.39079515e-02, 3.37984180e-05,
        7.30920812e-03, 9.46188255e-01, 5.25482718e-06],
       [1.86538035e-05, 2.79615816e-06, 3.40568865e-02, 9.34811277e-01,
        1.50466238e-02, 1.60341031e-02, 2.96594224e-05],
       [1.09323066e-02, 3.40497502e-02, 3.05540604e-01, 6.60901798e-03,
        5.66580634e-01, 6.87213267e-05, 7.62189659e-02],
       [4.48967205e-06, 3.67276045e-04, 9.04006272e-01, 2.41283576e-03,
        8.94706737e-02, 2.40961911e-03, 1.32883359e-03]])

In [41]:
lr.decision_function(test_scaled[:5])

array([[ -6.4981111 ,   1.03223154,   5.16362367,  -2.72862525,
          3.33884865,   0.32652003,  -0.63448754],
       [-10.85948496,   1.92717508,   4.77101291,  -2.3984238 ,
          2.97805242,   7.84135903,  -4.25969068],
       [ -4.33528506,  -6.23308874,   3.17445241,   6.48676482,
          2.35757378,   2.42113805,  -3.87155526],
       [ -0.68336294,   0.45273744,   2.64699742,  -1.18665018,
          3.26453415,  -5.75278095,   1.25852507],
       [ -6.39707605,  -1.99274198,   5.81573587,  -0.11029771,
          3.50281047,  -0.11163174,  -0.70679887]])

In [42]:
lr.coef_, lr.intercept_

(array([[-1.49001259, -1.02909653,  2.59342992,  7.70358183, -1.20070797],
        [ 0.19618191, -2.01072007, -3.77974344,  6.50492519, -1.99481478],
        [ 3.562807  ,  6.34355461, -8.48969786, -5.75757213,  3.79306162],
        [-0.10458363,  3.60320663,  3.93067948, -3.61740078, -1.7506979 ],
        [-1.40060998, -6.0750259 ,  5.25969918, -0.87219289,  1.86043812],
        [-1.38528461,  1.49213609,  1.39225441, -5.67734711, -4.40097409],
        [ 0.6215019 , -2.32405484, -0.9066217 ,  1.71600589,  3.69369499]]),
 array([-0.09204689, -0.26289902,  3.25100925, -0.14740759,  2.65498221,
        -6.78787045,  1.38423249]))

소프트맥스 함수
- 결정함수 값 : 다중 분류에서는, 총합이 1이 되도록 한다.

In [44]:
import numpy as np

decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]


In [45]:
from scipy.special import softmax

proba = softmax(decision, axis = 1)
print(np.round(proba, decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
